In [2]:
import os
import sys
root_dir = os.path.split(os.getcwd())[0]
project_dir = os.path.join(root_dir, 'python')
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [3]:
import subprocess
output = subprocess.Popen(f"cd {project_dir} && pipenv --venv", shell=True, stdout=subprocess.PIPE).stdout.read().decode('utf-8')
if len(output) > 0:
    # Include the pipenv in this notebook
    pipenv_path = os.path.join(output[:-1], 'lib', 'python3.6', 'site-packages')
    if pipenv_path not in sys.path:
        sys.path.append(pipenv_path)

In [4]:
url = "https://activepivot-ranch.activeviam.com:5700"
user = "admin"
password = "admin"

In [5]:
from pivot import Connector
from datetime import datetime

connector = Connector(url, user, password)

In [22]:
stores = connector.stores()
print(f"stores: {stores}")
print(f"forex: {connector.store_fields('Forex')})")

stores: ['City', 'CounterParty', 'Forex', 'Parameters', 'Product', 'Risk', 'Trade']
forex: {'Currency': <class 'str'>, 'TargetCurrency': <class 'str'>, 'Rate': None})


In [33]:
forex_ds = connector.store_query('Forex', fields = ['Currency', 'TargetCurrency', 'Rate'])
forex_ds.dataframe.head()

,Currency,Rate,TargetCurrency
0,USD,1.000000,USD
1,EUR,1.000000,EUR
2,EUR,1.467963,USD
3,USD,0.681216,EUR
4,CHF,1.000000,CHF


In [32]:
mdx_query = connector.mdx_query("""
SELECT 
NON EMPTY {[Measures].[pnl.SUM]} ON COLUMNS, 
NON EMPTY [Currency].[Currency].[Currency].Members ON ROWS 
FROM [EquityDerivativesCube]
""")
print(f'mdx query : {mdx_query.dataframe}')

mdx query :   Currency       pnl.SUM
0      EUR -4.014688e+07
1      GBP -3.073374e+06
2      USD -6.202210e+07
3      JPY -3.340923e+06
4      CHF -2.383163e+07
5      ZAR  4.064255e+07


In [49]:
real_pnl = mdx_query.dataframe.set_index('Currency').join(forex_ds.dataframe.set_index('Currency'))
real_pnl.head()

,pnl.SUM,Rate,TargetCurrency
Currency,,,
CHF,-2.383163e+07,1.000000,CHF
CHF,-2.383163e+07,0.576840,EUR
CHF,-2.383163e+07,0.846780,USD
CHF,-2.383163e+07,5.719819,ZAR
CHF,-2.383163e+07,0.294429,GBP


In [50]:
real_pnl['pnl.forex'] = real_pnl['pnl.SUM'] * real_pnl['Rate']
real_pnl.head()

,pnl.SUM,Rate,TargetCurrency,pnl.forex
Currency,,,,
CHF,-2.383163e+07,1.000000,CHF,-2.383163e+07
CHF,-2.383163e+07,0.576840,EUR,-1.374704e+07
CHF,-2.383163e+07,0.846780,USD,-2.018014e+07
CHF,-2.383163e+07,5.719819,ZAR,-1.363126e+08
CHF,-2.383163e+07,0.294429,GBP,-7.016714e+06


In [43]:
only_usd = real_pnl['Currency'] == 'USD'
real_pnl[only_usd][:5]

In [54]:
real_pnl.loc[['USD']]

,pnl.SUM,Rate,TargetCurrency,pnl.forex
Currency,,,,
USD,-6.202210e+07,1.000000,USD,-6.202210e+07
USD,-6.202210e+07,0.681216,EUR,-4.225046e+07
USD,-6.202210e+07,1.180945,CHF,-7.324466e+07
USD,-6.202210e+07,6.754790,ZAR,-4.189462e+08
USD,-6.202210e+07,76.522389,JPY,-4.746079e+09
USD,-6.202210e+07,0.347704,GBP,-2.156533e+07
